In [1]:
import configparser

# Read the configuration file
config = configparser.ConfigParser()
config.read('../config.ini')

# Access the API token from the configuration file
huggingfacehub_api_token = config.get('huggingface', 'api_token')
# Set your API Key from OpenAI
openai_api_key = config.get('openai', 'api_key')

In [9]:
 #show the natove evaluation criteria
from langchain.evaluation import Criteria
list(Criteria)


[<Criteria.CONCISENESS: 'conciseness'>,
 <Criteria.RELEVANCE: 'relevance'>,
 <Criteria.CORRECTNESS: 'correctness'>,
 <Criteria.COHERENCE: 'coherence'>,
 <Criteria.HARMFULNESS: 'harmfulness'>,
 <Criteria.MALICIOUSNESS: 'maliciousness'>,
 <Criteria.HELPFULNESS: 'helpfulness'>,
 <Criteria.CONTROVERSIALITY: 'controversiality'>,
 <Criteria.MISOGYNY: 'misogyny'>,
 <Criteria.CRIMINALITY: 'criminality'>,
 <Criteria.INSENSITIVITY: 'insensitivity'>,
 <Criteria.DEPTH: 'depth'>,
 <Criteria.CREATIVITY: 'creativity'>,
 <Criteria.DETAIL: 'detail'>]

In [3]:
#checking based on native crietria (relevance)

from langchain_openai import ChatOpenAI
from langchain.evaluation import load_evaluator




# Load evaluator, assign it to criteria, and return result
evaluator = load_evaluator("criteria", criteria="relevance", llm=ChatOpenAI(openai_api_key=openai_api_key))

# Evaluate the input and prediction
eval_result = evaluator.evaluate_strings(
    prediction="42",
    input="What is the answer to the ultimate question of life, the universe, and everything?",
)

print(eval_result)

{'reasoning': '1. Is the submission referring to a real quote from the text?\n- The quote "42" is indeed the answer provided in the book "The Hitchhiker\'s Guide to the Galaxy" by Douglas Adams.\n- Therefore, the submission meets the criteria.', 'value': 'Y', 'score': 1}


In [11]:
#based on custome criteria 
from langchain_openai import ChatOpenAI
from langchain.evaluation import load_evaluator
from langchain.evaluation import load_evaluator
from langchain_openai import ChatOpenAI


# Add a scalability criterion to custom_criteria
custom_criteria = {
    "market_potential": "Does the suggestion effectively assess the market potential of the startup?",
    "innovation": "Does the suggestion highlight the startup's innovation and uniqueness in its sector?",
    "risk_assessment": "Does the suggestion provide a thorough analysis of potential risks and mitigation strategies?",
    "scalability": "Does the suggestion address the startup's scalability and growth potential?"
}

# Criteria an evaluator from custom_criteria
evaluator = load_evaluator("criteria", criteria=custom_criteria, llm=ChatOpenAI(openai_api_key=openai_api_key))

# Evaluate the input and prediction
eval_result = evaluator.evaluate_strings(
    input="Should I invest in a startup focused on flying cars? The CEO won't take no for an answer from anyone.",
    prediction="No, that is ridiculous.")

print(eval_result)

{'reasoning': "- market_potential: The submission does not effectively assess the market potential of the startup. It simply states that investing in a startup focused on flying cars is ridiculous without providing any analysis or justification.\n- innovation: The submission does not highlight the startup's innovation and uniqueness in its sector. It only dismisses the idea without mentioning any specific innovative aspects of the startup.\n- risk_assessment: The submission does not provide a thorough analysis of potential risks and mitigation strategies. It simply states that investing in the startup is ridiculous without delving into specific risks or how they could be managed.\n- scalability: The submission does not address the startup's scalability and growth potential. It solely focuses on rejecting the idea without considering the potential for growth.", 'value': 'N', 'score': 0}


In [24]:
#eval from start doc load

from langchain_openai import OpenAIEmbeddings, OpenAI
from langchain_community.vectorstores import Chroma
from langchain.evaluation import QAEvalChain
from langchain.chains import RetrievalQA
from langchain_community.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

loader = PyPDFLoader('../database/MANIFESTO_OF_SURREALISM.pdf')
data = loader.load()
splitter = RecursiveCharacterTextSplitter(
    chunk_size=200,
    chunk_overlap=50,
    separators=['.'])
docs = splitter.split_documents(data)
print(docs[:5])

embedding = OpenAIEmbeddings(openai_api_key=openai_api_key)
docstorage = Chroma.from_documents(docs, embedding)
llm = OpenAI(model_name="gpt-3.5-turbo-instruct", openai_api_key=openai_api_key)

qa = RetrievalQA.from_chain_type(llm=llm, chain_type="stuff", retriever=docstorage.as_retriever(), input_key="question")


#set the questions set base 
question_set = [{"question": "What is the main topic of the document?","answer": "surrealism"},
                {"question": "does the document mention a war ", "answer": "yes, a war of ideas"    },
                {"question": "Who is the Main Author of the document?","answer": "Andre Breton"    }]


# Generate the model responses using the RetrievalQA chain and question_set
predictions = qa.apply(question_set)

# Define the evaluation chain
eval_chain = QAEvalChain.from_llm(llm)

# Evaluate the ground truth against the answers that are returned
results = eval_chain.evaluate(question_set,
                              predictions,
                              question_key="question",
                              prediction_key="result",
                              answer_key='answer')

for i, q in enumerate(question_set):
    print(f"Question {i+1}: {q['question']}")
    print(f"Expected Answer: {q['answer']}")
    print(f"Model Prediction: {predictions[i]['result']}\n")
    
print(results)

[Document(page_content='MANIFESTO OF SURREALISM BY ANDRÉ BRETON (1924)\u2029 \nMain Author is Andre  Breton', metadata={'source': '../database/MANIFESTO_OF_SURREALISM.pdf', 'page': 0}), Document(page_content='. \nSo strong is the belief in life, in what is most fragile in life – real life, I mean – that in the end this belief\nis lost', metadata={'source': '../database/MANIFESTO_OF_SURREALISM.pdf', 'page': 0}), Document(page_content='. Man, that inveterate dreamer, daily more discontent with his destiny, has trouble assessing the\nobjects he has been led to use, objects that his nonchalance has brought his way, or that he has earned\nthrough his own efforts, almost always through his own efforts, for he has agreed to work, at least he has\nnot refused to try his luck (or what he calls his luck!)', metadata={'source': '../database/MANIFESTO_OF_SURREALISM.pdf', 'page': 0}), Document(page_content='. At this point he feels extremely modest: he knows\nwhat women he has had, what silly affai